## Aufgabe 16: Lösungsvorschlag

In [1]:
using LinearAlgebra

### (c) 

Um den Rückwärtsfehler $\omega$ zu berechnen, verwenden wir den Satz von Rigal-Gaches [B, §15.2]:

In [2]:
# Rückwärtsfehler in x nach Rigal-Gaches, numerisch aber mindestens die Maschinengenauigkeit

ω(A,b,x) = max(norm(b - A*x,Inf)/(opnorm(A,Inf)*norm(x,Inf)), eps()/2); 

Wir konstruieren eine schlecht konditionierte $2 \times 2$ Matrix mit fast linear abhängigen Zeilen, achten aber darauf, keine einfachen Zahlen wie $1$, $2$, etc. zu verwenden. Damit erhalten wir (achte auf die Wiederverwendung der $PLR$-Zerlegung `F` von $A$) z.B.:

In [3]:
m = 2
A = 3.0*[1 1; 1-1e-14 1]

ALR = copy(A)
F = lu!(ALR)   # zur Wiederverwendung der Dreieckszerlegung von A
@show κ = 1/LAPACK.gecon!('I', ALR, opnorm(A, Inf)) # sehr schlecht konditioniert, aber nicht numerisch singulär

b = ones(m,1)
x_std = F\b; 
x_Kunstfehler = inv(F)*b

ω_Backslash = ω(A,b,x_std)            # Rückwärtsfehler für x = A\b
ω_Kunstfehler = ω(A,b,x_Kunstfehler)  # Rückwärtsfehler für x = inv(A)*b

println("Dimension = ",m,"; ω_Backslash = ",ω_Backslash,"; ω_Kunstfehler = ",ω_Kunstfehler)

κ = 1 / LAPACK.gecon!('I', ALR, opnorm(A, Inf)) = 3.973764377091614e14
Dimension = 2; ω_Backslash = 1.1102230246251565e-16; ω_Kunstfehler = 0.001937984496124031


Das Ergebnis des Kunstfehlers besitzt einen Rückwärtsfehler $\omega \gg \epsilon_\text{mach}$, ist also **nicht** rückwärtsstabil.

### (d) 

Wir schätzen die Vorwärtsfehler über die Formel aus [B, §15.3] ab:

$$
  \text{Vorwärtsfehler}\le\kappa(A)\cdot\text{Rückwärtsfehler}
$$

In [4]:
η_Backslash = κ*ω_Backslash     # Vorwärtsfehlerschätzung für x = A\b
η_Kunstfehler = κ*ω_Kunstfehler # Vorwärtsfehlerschätzung für x = inv(A)*b

η_Backslash, η_Kunstfehler

(0.044117647058823525, 7.701093754053514e11)

Da wir die _Abschätzung_ des Vorwärtsfehler immer mit dem *unvermeidbaren Vorwärtsfehler* $\kappa(A)\epsilon_\text{mach}$ vergleichen müssen, fällt die Beurteilung natürlich genau wie beim Rückwärtsfehler aus (die Konditionszahl kürzt sich heraus):
die $PLR$-Zerlegung liefert ein stabiles, der Kunstfehler ein instabiles Resultat. 

Es ist aber davon auszugehen, dass insbesondere die zweite Abschätzung den Vorwärtsfehler beträchtlich _überschätzt_. Einen Hinweis hierfür liefert der unmittelbare Vergleich der beiden Lösungen:

In [5]:
norm(x_std - x_Kunstfehler,Inf)/norm(x_std,Inf)

0.008012820512820526